# Experiment: antonym-negation2

Train a model first on the Checklist-style antonym-negation-degree dataset.
Then continue with training on SQuad.

Augment dataset with arbitrary text from Wikipedia.

Does this additional training imrpove performance on Checklist, while hopefully imrpoving performance on Squad as well?


In [1]:
cd ../..

/home/sambeck/code/nlpfp


In [2]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser, pipeline
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json

NUM_PREPROCESSING_WORKERS = 2

In [3]:
MODEL='google/electra-small-discriminator'
TASK='qa'
DATASET='squad'
MAX_LENGTH=128
TRAIN_MAX_SAMPLES=None

### Train on Checklist antonym-negation-degree dataset first

(you can make a basic Checklist dataset using the make_checklist_dataset notebook)

In [4]:
KEYS = ['id', 'title', 'context', 'question', 'answers']

In [5]:
dataset = datasets.Dataset.from_file('./experiments/antonym-negation2/dataset.arrow')
dataset = dataset.shuffle()
dataset = dataset.train_test_split(test_size=0.1)

In [6]:
sqd = datasets.load_dataset('squad')

Reusing dataset squad (/home/sambeck/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [11]:
lumped_data = {}
for key in KEYS:
    lumped_data[key] = sqd['train'][key]
    lumped_data[key].extend(sqd['validation'][key])
    lumped_data[key].extend(dataset['train'][key])
    lumped_data[key].extend(dataset['test'][key])
    

In [15]:
dataset = datasets.Dataset.from_dict(lumped_data)

In [16]:
dataset = dataset.shuffle()
dataset = dataset.train_test_split(test_size=0.05)


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 199321
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10491
    })
})

### Model

In [18]:
model_class = AutoModelForQuestionAnswering
# Initialize the model and tokenizer from the specified pretrained model/checkpoint
model = model_class.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.

In [19]:
# model pipeline (includes tokenization)
mp = pipeline("question-answering", tokenizer=tokenizer, model=model, device=0)

In [20]:
pred = mp(question="Which color is the dog?", context="There is a black dog.", truncation=True, )
print(pred)

{'score': 0.029421012848615646, 'start': 9, 'end': 10, 'answer': 'a'}


### Set up dataset for training

In [21]:
train_dataset_featurized = None
eval_dataset_featurized = None
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [22]:
prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)

if TRAIN_MAX_SAMPLES:
    train_dataset = train_dataset.select(range(TRAIN_MAX_SAMPLES))

print('featurize train...')
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
)

print('featurize test...')
eval_dataset_featurized = eval_dataset.map(
    prepare_eval_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=eval_dataset.column_names
)


featurize train...
featurize test...


In [23]:
trainer_class = Trainer
eval_kwargs = {}
# If you want to use custom metrics, you should define your own "compute_metrics" function.
# For an example of a valid compute_metrics function, see compute_accuracy in helpers.py.
compute_metrics = None
# For QA, we need to use a tweaked version of the Trainer (defined in helpers.py)
# to enable the question-answering specific evaluation metrics
trainer_class = QuestionAnsweringTrainer
eval_kwargs['eval_examples'] = eval_dataset
metric = datasets.load_metric('squad')
compute_metrics = lambda eval_preds: metric.compute(
    predictions=eval_preds.predictions, references=eval_preds.label_ids)

In [24]:
# This function wraps the compute_metrics function, storing the model's predictions
# so that they can be dumped along with the computed metrics
eval_predictions = None
def compute_metrics_and_store_predictions(eval_preds):
    global eval_predictions
    eval_predictions = eval_preds
    return compute_metrics(eval_preds)

### Train

In [25]:
# Initialize the Trainer object with the specified arguments and the model and dataset we loaded above
trainer = trainer_class(
    model=model,
    train_dataset=train_dataset_featurized,
    eval_dataset=eval_dataset_featurized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_and_store_predictions
)

In [26]:
trainer.train()

***** Running training *****
  Num examples = 200042
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 75018


Step,Training Loss
500,3.120400
1000,2.172400
1500,1.780800
2000,1.613600
2500,1.482300
3000,1.477000
3500,1.386800
4000,1.328400
4500,1.359300
5000,1.322900


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1500
Configuration saved in tmp_trainer/checkpoint-1500/config.json
Model weights saved in tmp_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in tmp_traine

tokenizer config file saved in tmp_trainer/checkpoint-12000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-12000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-12500
Configuration saved in tmp_trainer/checkpoint-12500/config.json
Model weights saved in tmp_trainer/checkpoint-12500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-12500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-12500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-13000
Configuration saved in tmp_trainer/checkpoint-13000/config.json
Model weights saved in tmp_trainer/checkpoint-13000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-13000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-13000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-13500
Configuration saved in tmp_trainer/checkpoint-13500/config.json
Model we

Special tokens file saved in tmp_trainer/checkpoint-23500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-24000
Configuration saved in tmp_trainer/checkpoint-24000/config.json
Model weights saved in tmp_trainer/checkpoint-24000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-24000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-24000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-24500
Configuration saved in tmp_trainer/checkpoint-24500/config.json
Model weights saved in tmp_trainer/checkpoint-24500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-24500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-24500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-25000
Configuration saved in tmp_trainer/checkpoint-25000/config.json
Model weights saved in tmp_trainer/checkpoint-25000/pytorch_model.bin
tokenizer config fil

Saving model checkpoint to tmp_trainer/checkpoint-35500
Configuration saved in tmp_trainer/checkpoint-35500/config.json
Model weights saved in tmp_trainer/checkpoint-35500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-35500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-35500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-36000
Configuration saved in tmp_trainer/checkpoint-36000/config.json
Model weights saved in tmp_trainer/checkpoint-36000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-36000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-36000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-36500
Configuration saved in tmp_trainer/checkpoint-36500/config.json
Model weights saved in tmp_trainer/checkpoint-36500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-36500/tokenizer_config.json
Special tokens file 

Configuration saved in tmp_trainer/checkpoint-47000/config.json
Model weights saved in tmp_trainer/checkpoint-47000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-47000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-47000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-47500
Configuration saved in tmp_trainer/checkpoint-47500/config.json
Model weights saved in tmp_trainer/checkpoint-47500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-47500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-47500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-48000
Configuration saved in tmp_trainer/checkpoint-48000/config.json
Model weights saved in tmp_trainer/checkpoint-48000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-48000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-48000/special_tokens_map

Model weights saved in tmp_trainer/checkpoint-58500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-58500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-58500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-59000
Configuration saved in tmp_trainer/checkpoint-59000/config.json
Model weights saved in tmp_trainer/checkpoint-59000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-59000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-59000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-59500
Configuration saved in tmp_trainer/checkpoint-59500/config.json
Model weights saved in tmp_trainer/checkpoint-59500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-59500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-59500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-60000
Co

tokenizer config file saved in tmp_trainer/checkpoint-70000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-70000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-70500
Configuration saved in tmp_trainer/checkpoint-70500/config.json
Model weights saved in tmp_trainer/checkpoint-70500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-70500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-70500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-71000
Configuration saved in tmp_trainer/checkpoint-71000/config.json
Model weights saved in tmp_trainer/checkpoint-71000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-71000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-71000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-71500
Configuration saved in tmp_trainer/checkpoint-71500/config.json
Model we

TrainOutput(global_step=75018, training_loss=0.8035212766612811, metrics={'train_runtime': 22160.2767, 'train_samples_per_second': 27.081, 'train_steps_per_second': 3.385, 'total_flos': 1.7534198093303808e+16, 'train_loss': 0.8035212766612811, 'epoch': 3.0})

In [27]:
pred = mp(question="Which color is the dog?", context="There is a black dog.", truncation=True, )
print(pred)

{'score': 0.9935815334320068, 'start': 11, 'end': 16, 'answer': 'black'}


In [28]:
pred = mp(question='Who is the most awesome?', context='William is awesome, but John is more awesome', truncation=True, )
print(pred)

{'score': 0.9998321533203125, 'start': 24, 'end': 28, 'answer': 'John'}


In [29]:
pred = mp(question='Who is the least awesome?', context='William is awesome, but John is more awesome', truncation=True, )
print(pred)

{'score': 0.9891671538352966, 'start': 0, 'end': 7, 'answer': 'William'}


In [30]:
pred = mp(question='Who is less awesome?', context='William is awesome, but John is more awesome', truncation=True, )
print(pred)

{'score': 0.8351035118103027, 'start': 24, 'end': 28, 'answer': 'John'}


In [31]:
pred = mp(question="Which thing is hot?", context="There is a cold gopher, a polar bear, and a hot snake.", truncation=True, )
print(pred)

{'score': 0.9975577592849731, 'start': 48, 'end': 53, 'answer': 'snake'}


In [32]:
pred = mp(question="Which thing is least hot?", context="There is a cold gopher, a polar bear, and a hot snake.", truncation=True, )
print(pred)

{'score': 0.3253280520439148, 'start': 44, 'end': 53, 'answer': 'hot snake'}


In [33]:
pred = mp(question="Who is most hot?", context="John is not cold. Bob is cold.", truncation=True, )
print(pred)

{'score': 0.9983034133911133, 'start': 18, 'end': 21, 'answer': 'Bob'}


# retrain on Squad

In [34]:
pwd

'/home/sambeck/code/nlpfp'

In [35]:
trainer.save_model("./experiments/antonym-negation2/model")

Saving model checkpoint to ./experiments/antonym-negation2/model
Configuration saved in ./experiments/antonym-negation2/model/config.json
Model weights saved in ./experiments/antonym-negation2/model/pytorch_model.bin
tokenizer config file saved in ./experiments/antonym-negation2/model/tokenizer_config.json
Special tokens file saved in ./experiments/antonym-negation2/model/special_tokens_map.json


In [36]:
train_dataset[:10]

OrderedDict([('id',
              ['56fb896db28b3419009f1e1a',
               '573215afb9d445190005e7c5',
               '57270817708984140094d8c9',
               '572aaec2111d821400f38cb2',
               '570cd2e4b3d812140066d295',
               '5726f90b708984140094d760',
               'x1b429523645616ed',
               '5726d77f708984140094d321',
               '570b745a6b8089140040f975',
               '5726c667f1498d1400e8eaf1']),
             ('title',
              ['Middle_Ages',
               'Bird',
               'Victoria_and_Albert_Museum',
               'Royal_assent',
               'Macintosh',
               'Victoria_and_Albert_Museum',
               'goldilocks',
               'Nigeria',
               'Hard_rock',
               'Queen_(band)']),
             ('context',
              ['Strong, royalty-based nation states rose throughout Europe in the Late Middle Ages, particularly in England, France, and the Christian kingdoms of the Iberian Peninsula: Ara